# コードジェネレーター

要件：オープンソースモデルを使用して、Pythonコードから高性能C++コードを生成する。

これを複製するには、ビデオ（例えば[コレ](https://www.youtube.com/watch?v=IC9n8nReeUY)とか？）を参考に、`Qwen/CodeQwen1.5-7B-Chat`、`google/codegemma-7b-it`のHugging Faceエンドポイントをセットアップします。それは簡単であり、結果を見るのは非常に満足です！それはあなたの学習の重要な部分でもあります。これは、APIの背後にあるOSSのLLMを展開する最初の例です。これは8週目に戻りますが、これはOSSのLLMを生産に移動することに関与するもののためにあなたの心に種を植えるべきです。

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h1 style="color:#900;">重要 - 使用していないエンドポイントは一時停止してください</h1>
            <span style="color:#900;">
            このプロジェクトでHuggingFaceエンドポイントを使用する場合は、不要な実行コストが発生しないように、作業が完了したらエンドポイントを停止または一時停止してください。エンドポイントは使用時のみ実行すれば、コストは非常に低くなります。<a href="https://ui.endpoints.huggingface.co/">ココ</a>からHuggingFaceエンドポイントUIにアクセスし、エンドポイントを開いて「一時停止」をクリックすると一時停止状態になり、それ以降は料金が発生しなくなります。この点についてご指摘いただいた学生のJohn L.に感謝します。<br/><br/>第8週では、HuggingFaceエンドポイントの代わりにModalを使用します。Modalでは、使用した時間に対してのみ料金が発生し、無料クレジットが付与されます。
            </span>
        </td>
    </tr>
</table>

ここでは、課金が必要な Anthropic の Claude も HuggingFace エンドポイントも使用していない。また、day3_ja.ipynbと重複する余分なセル / コードは削除してある。しかし、ソレ以外の所は動くようにしてあるので、必要に応じて、Claude も `Qwen/CodeQwen1.5-7B-Chat`、`google/codegemma-7b-it` の HuggingFace エンドポイントを試すことは出来る。

In [1]:
# import

# 基本
import os
import io
import sys
import subprocess

# REST
import json
import requests

# WebAPI+Key
from dotenv import load_dotenv
from openai import OpenAI
#import google.generativeai
#import anthropic

# 出力、WebUI
from IPython.display import Markdown, display, update_display
import gradio as gr

In [2]:
# .envファイルから環境変数をロード

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
#os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
#os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [3]:
# 初期化

openai = OpenAI()
#claude = anthropic.Anthropic()

OPENAI_MODEL = "gpt-4.1"
#CLAUDE_MODEL = "claude-3-5-sonnet-20240620"

In [4]:
# system_message
system_message = "You are an assistant that reimplements Python code in high performance C++ for an Ubuntu. \
Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. \
The C++ response needs to produce an identical output in the fastest possible time."
# "あなたは、Ubuntu 向けに高性能 C++ で Python コードを再実装するアシスタントです。\
# C++ コードのみで応答してください。コメントは控えめに使用し、時折コメントする以外は説明は提供しないでください。\
# C++ の応答では、可能な限り最短時間で同一の出力を生成する必要があります。"

In [5]:
# user_prompt
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. \
    Respond only with C++ code; do not explain your work other than a few comments. \
    Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    # "このPythonコードを、最短時間で同一の出力を生成する、可能な限り高速な実装でC++に書き直してください。\
    # C++コードのみで回答してください。作業内容については、コメント以外に説明は不要です。\
    # intオーバーフローが発生しないように、数値型に注意してください。iomanipなどの必要なC++パッケージはすべて#includeを忘れないでください。\n\n"
    user_prompt += python
    return user_prompt

In [6]:
# messages = system_message + user_prompt_for(x)
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [7]:
code_name = "optimize_y_pg"

In [8]:
# *.cppファイルに書き込みます
def write_output(cpp, name):
    code = cpp.replace("```cpp","").replace("```","")
    with open(name + ".cpp", "w") as f:
        f.write(code)

In [9]:
# πの近似計算（数値級数）
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [10]:
# 擬似乱数を使って生成された整数配列に対し、最大部分配列和（Maximum Subarray Sum）を複数回（20回）計算し、その合計を出力する。

# lcg：線形合同法により、一定の周期で乱数を生成する擬似乱数生成器
# max_subarray_sum：lcgで長さnの乱数配列を生成し最大部分配列和を求める
# total_max_subarray_sum：長さ10000の乱数配列から最大部分配列和を求める処理を20回繰り返し合計

python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

# Gradio

In [11]:
import gradio as gr

In [12]:
# optimize_gptのGUI版
def stream_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [ ]:
# optimize_claudeのGUI版
def stream_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```cpp\n','').replace('```','')

In [13]:
# 入力されたpythonコードを実行する
def execute_python(code):
    # print(code)
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code, {}) # 危険（codeに攻撃者の入力が渡らないように注意 {}はスコープ一貫性
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [14]:
# プラットフォームのC++コードをコンパイルするために、トライブロックのコードを変更
# 最適化されたC++コードをコンパイルおよび実行するUbuntuバージョン：
def execute_cpp(code):
    write_output(code, code_name)
    try:
        # M1 Mac
        #compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m1", "-mcpu=apple-m1", "-o", "optimize_x_pg", "optimize_x_pg.cpp"]
        # Ubuntu
        compile_cmd = ["g++", "-O2", "-o", code_name, code_name + ".cpp"]
        compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
        run_cmd = ["./" + code_name]
        run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
        return run_result.stdout
    except subprocess.CalledProcessError as e:
        return f"An error occurred:\n{e.stderr}"

In [15]:
css = """
.python {background-color: # 306998;}
.cpp {background-color: # 050;}
"""

# OSSのLLMを追加

In [ ]:
from huggingface_hub import login, InferenceClient
from transformers import AutoTokenizer

In [ ]:
# Hugging Faceにログイン

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

In [ ]:
# Hugging Face BigCodeよりスコアが高いものを選択
code_qwen = "Qwen/CodeQwen1.5-7B-Chat"
code_gemma = "google/codegemma-7b-it"
# Hugging Faceエンドポイントをセットアップ
CODE_QWEN_URL = "https://h1vdol7jxhje3mpn.us-east-1.aws.endpoints.huggingface.cloud"
CODE_GEMMA_URL = "https://c5hggiyqachmgnqg.us-east-1.aws.endpoints.huggingface.cloud"

In [ ]:
# optimize_code_qwen(ないケド笑)のGUI版
def stream_code_qwen(python):
    tokenizer = AutoTokenizer.from_pretrained(code_qwen)
    messages = messages_for(python)
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    client = InferenceClient(CODE_QWEN_URL, token=hf_token)
    stream = client.text_generation(text, stream=True, details=True, max_new_tokens=3000)
    result = ""
    for r in stream:
        result += r.token.text
        yield result    

In [16]:
# Gradioから実行する際のコントローラ
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    #elif model=="Claude":
    #    result = stream_claude(python)
    #elif model=="CodeQwen":
    #    result = stream_code_qwen(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far    

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">素晴らしい貢献をしてくださった@CloudLlamaさん、ありがとうございます。</h2>
            <span style="color:#090;">
                学生がコードの一部を寄稿し、この問題を解決しました。次の2つのセルをご覧ください。実行するPythonプログラムを選択できるようになり、PC、Windows、Macで動作するコンパイラが自動的に選択されるようになりました。@CloudLlamaさん、本当にありがとうございます！
            </span>
        </td>
    </tr>
</table>

In [17]:
# コンボ
def select_sample_program(sample_program):
    if sample_program=="pi":
        return pi
    elif sample_program=="python_hard":
        return python_hard
    else:
        return "Type your Python program here"

In [18]:
import platform

VISUAL_STUDIO_2022_TOOLS = "C:\\Program Files\\Microsoft Visual Studio\\2022\\Community\\Common7\Tools\\VsDevCmd.bat"
VISUAL_STUDIO_2019_TOOLS = "C:\\Program Files (x86)\\Microsoft Visual Studio\\2019\\BuildTools\\Common7\\Tools\\VsDevCmd.bat"

simple_cpp = """
#include <iostream>

int main() {
    std::cout << "Hello";
    return 0;
}
"""

def run_cmd(command_to_run):
    try:
        run_result = subprocess.run(command_to_run, check=True, text=True, capture_output=True)
        return run_result.stdout if run_result.stdout else "SUCCESS"
    except:
        return ""

def c_compiler_cmd(filename_base):
    my_platform = platform.system()
    my_compiler = []

    try:
        with open("simple.cpp", "w") as f:
            f.write(simple_cpp)
            
        if my_platform == "Windows":
            if os.path.isfile(VISUAL_STUDIO_2022_TOOLS):
                if os.path.isfile("./simple.exe"):
                    os.remove("./simple.exe")
                compile_cmd = ["cmd", "/c", VISUAL_STUDIO_2022_TOOLS, "&", "cl", "simple.cpp"]
                if run_cmd(compile_cmd):
                    if run_cmd(["./simple.exe"]) == "Hello":
                        my_compiler = ["Windows", "Visual Studio 2022", ["cmd", "/c", VISUAL_STUDIO_2022_TOOLS, "&", "cl", f"{filename_base}.cpp"]]
        
            if not my_compiler:
                if os.path.isfile(VISUAL_STUDIO_2019_TOOLS):
                    if os.path.isfile("./simple.exe"):
                        os.remove("./simple.exe")
                    compile_cmd = ["cmd", "/c", VISUAL_STUDIO_2019_TOOLS, "&", "cl", "simple.cpp"]
                    if run_cmd(compile_cmd):
                        if run_cmd(["./simple.exe"]) == "Hello":
                            my_compiler = ["Windows", "Visual Studio 2019", ["cmd", "/c", VISUAL_STUDIO_2019_TOOLS, "&", "cl", f"{filename_base}.cpp"]]
    
            if not my_compiler:
                my_compiler=[my_platform, "Unavailable", []]
                
        elif my_platform == "Linux":
            if os.path.isfile("./simple"):
                os.remove("./simple")
            compile_cmd = ["g++", "simple.cpp", "-o", "simple"]
            if run_cmd(compile_cmd):
                if run_cmd(["./simple"]) == "Hello":
                    my_compiler = ["Linux", "GCC (g++)", ["g++", f"{filename_base}.cpp", "-o", f"{filename_base}" ]]
    
            if not my_compiler:
                if os.path.isfile("./simple"):
                    os.remove("./simple")
                compile_cmd = ["clang++", "simple.cpp", "-o", "simple"]
                if run_cmd(compile_cmd):
                    if run_cmd(["./simple"]) == "Hello":
                        my_compiler = ["Linux", "Clang++", ["clang++", f"{filename_base}.cpp", "-o", f"{filename_base}"]]
        
            if not my_compiler:
                my_compiler=[my_platform, "Unavailable", []]
    
        elif my_platform == "Darwin":
            if os.path.isfile("./simple"):
                os.remove("./simple")
            compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m1", "-mcpu=apple-m1", "-o", "simple", "simple.cpp"]
            if run_cmd(compile_cmd):
                if run_cmd(["./simple"]) == "Hello":
                    my_compiler = ["Macintosh", "Clang++", ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m1", "-mcpu=apple-m1", "-o", f"{filename_base}", f"{filename_base}.cpp"]]
    
            if not my_compiler:
                my_compiler=[my_platform, "Unavailable", []]
    except:
        my_compiler=[my_platform, "Unavailable", []]
        
    if my_compiler:
        return my_compiler
    else:
        return ["Unknown", "Unavailable", []]


<>:3: SyntaxWarning: invalid escape sequence '\T'
<>:3: SyntaxWarning: invalid escape sequence '\T'
/tmp/ipykernel_24251/519549262.py:3: SyntaxWarning: invalid escape sequence '\T'
  VISUAL_STUDIO_2022_TOOLS = "C:\\Program Files\\Microsoft Visual Studio\\2022\\Community\\Common7\Tools\\VsDevCmd.bat"


In [19]:
compiler_cmd = c_compiler_cmd(code_name)

with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=python_hard, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        with gr.Column():
            sample_program = gr.Radio(["pi", "python_hard"], label="Sample program", value="python_hard")
            model = gr.Dropdown(["GPT", "Claude", "CodeQwen"], label="Select model", value="GPT")
        with gr.Column():
            architecture = gr.Radio([compiler_cmd[0]], label="Architecture", interactive=False, value=compiler_cmd[0])
            compiler = gr.Radio([compiler_cmd[1]], label="Compiler", interactive=False, value=compiler_cmd[1])
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        if not compiler_cmd[1] == "Unavailable":
            cpp_run = gr.Button("Run C++")
        else:
            cpp_run = gr.Button("No compiler to run C++", interactive=False)
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    sample_program.change(select_sample_program, inputs=[sample_program], outputs=[python])
    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.


gio: http://127.0.0.1:7870/: Operation not supported
